# How to compose a timing model component

## Building the timing model component from scratch

This example notebook includes the following contents
* Defining a timing model component class
  * Necessary parts
  * Conventions
* Use it with the `TimingModel` class
  * Add the new component to the `TimingModel` class
  * Use the functions in the `TimingModel` class to interact with the new component.
  
We will build a simple model component, pulsar spindow model with spin period as parameters, instead of spin frequency. 

## Import the necessary modules

In [3]:
import numpy as np   # Numpy is a widely used package
# PINT uses astropy units in the internal cacluation and is highly recommended for a new component
import astropy.units as u  
# Import the component classes. 
from pint.models.timing_model import TimingModel, Component, PhaseComponent, Missing
import pint.models.parameter as p 

## Define the timing model class

A timing model component should be an inheritance/subclass of `pint.models.timing_model.Component`. PINT also pre-defines three component subclasses for the most used type of components and they have different attribute and functions (see: https://nanograv-pint.readthedocs.io/en/latest/api/pint.models.timing_model.html):
* DelayComponent for delay type of models. 
* PhaseComponent for phase type of models.
* NoiseComponent for noise type of models.

Here since we are making a spin-down model, we will use the `PhaseComponent`.

### Required parts
* Model parameters, generally defined as `PINT.models.parameter.Parameter` class or its subclasses. (see https://nanograv-pint.readthedocs.io/en/latest/api/pint.models.parameter.html)
* Model functions, defined as methods in the component, including:
    * .setup(), for setting up the component(e.g., registering the derivatives). 
    * .validate(), for checking if the parameters have the correct inputs. 
    * Modeled quantity functions.
    * The derivative of modeled quantities.
    * Other support functions. 

### Conventions

To 
* Set the class attribute `.register` to be True so that it is in the searching space of model builder 
* Add the method to computing model quantity in the list 
* 

In [ ]:
class PeriodSpindown(PhaseComponent):
    """This is an example model component of pular spindown but parametrized as period. 
    """
    register = True # Flags for the model builder to find this component.
    # define the init function.
    # Most components do not have a parameter for input.
    def __init__(self): 
        # Get the attruibutes that initilzed in the parent class
        super().__init__()
        # Add parameters using the add_params in the TimingModel 
        # Add spin period as parameter
        self.add_param(p.floatParameter(name='P0', value=None, units=u.s, 
                                        description="Spin period", longdouble=True))
        # Add spin period derivative 
        self.add_param(p.floatParameter(name='P1', value=None, units=u.s/u.s, 
                                        description="Spin period derivative", longdouble=True))
        # Add reference epoch time
        self.add_param(p.MJDParameter(name="PEPOCH", description="Reference epoch for spin-down", 
                                      time_scale='tdb'))
        # Add spindown phase model function to phase functions
        self.phase_funcs_component += [self.spindown_phase_period] 
        # Add the d_phase_d_delay derivative to the list
        self.phase_derive_wrt_delay += [self.d_spindown_phase_period_d_delay]
        
    def setup(self):
        """Setup the model. Register the derivative functions"""
        super().setup()
        self.register_deriv_funcs(self.d_phase_d_P0, 'P0')
        self.register_deriv_funcs(self.d_phase_d_P1, 'P1')
            
    def validate(self):
        """Check the parameter value."""
        super().validate()
        for param in ['P0', 'P1']:
            if getattr(self, param) is None:
                raise ValueError("Spindown period model needs {}".format(param))
        
    def get_dt(self, toa, delay):
        pass
    
    def spindown_phase(self, toas, delay):
        pass 
    
    def d_spindown_phase_period_d_delay(self, toas, delay):
        pass
    
    def d_phase_d_P0(self, toas, param, delay):
        pass
    
    def d_phase_d_P1(self, toas, param, delay):
        pass
    